In [91]:
import pandas as pd
import matplotlib.pyplot as plt 
from datetime import datetime  
from nltk.tokenize import word_tokenize  

from nltk.tokenize import TweetTokenizer
# tknzr = TweetTokenizer()
# tknzr.tokenize("The code didn't work!")
# [u'The', u'code', u"didn't", u'work', u'!']
from gensim.corpora import Dictionary
import itertools
from collections import defaultdict
# from wordcloud import WordCloud



In [92]:
df = pd.read_csv('../data/processed/cleaned_data_2.csv')
# increasing max length for all columns
pd.set_option('display.max_colwidth', -1)

In [88]:
# functions for tokenizing, word count and wordcloud
def tokenize(s):
    """
    Tokenize each text into a list of words
    """
    tokens = word_tokenize(s)
    return [w for w in tokens if w.isalpha()]


def get_tokens_frequency_df(series):
    """
    Count each time the same word appeared in the series
    :return: dataFrame
    """
    corpus_lists = [doc for doc in series.dropna() if doc]
    dictionary = Dictionary(corpus_lists)
    corpus_bow = [dictionary.doc2bow(doc) for doc in corpus_lists]
    token_freq_bow = defaultdict(int)
    for token_id, token_sum in itertools.chain.from_iterable(corpus_bow):
        token_freq_bow[token_id] += token_sum

    return pd.DataFrame(list(token_freq_bow.items()), columns=['token_id', 'token_count']).assign(
        token=lambda df1: df1.apply(lambda df2: dictionary.get(df2.token_id), axis=1),
        doc_appeared=lambda df1: df1.apply(lambda df2: dictionary.dfs[df2.token_id], axis=1)).reindex(
        labels=['token_id', 'token', 'token_count', 'doc_appeared'], axis=1).set_index('token_id')


def plot_word_cloud(df, top_n, save_as):
    word_cloud = WordCloud(background_color='white', colormap='magma', contour_width=1,
                           contour_color='orange', relative_scaling=0.5)

    sorted_freq_dict = dict(df[['token', 'token_count']].nlargest(top_n, columns='token_count').values)
    wc = word_cloud.generate_from_frequencies(frequencies=sorted_freq_dict, max_font_size=40)

    _, ax = plt.subplots(figsize=(15, 8))
    ax.set_title('Term Frequency', fontsize=16)

    ax.imshow(wc, interpolation='bilinear')
    ax.axis('off')

    plt.savefig(save_as, orientation='portrait', format='png')

In [94]:
# for some reason we keep having missing values....
df.isna().sum()

tweet_low                          0 
handle_count                       0 
no_handle_no_special_no_sin_ash    0 
tweet_length                       0 
character_count                    0 
lemmatized                         4 
lemmatized_1                       4 
lemmatized_final                   4 
tweet_without_stopwords            27
tweet_without_stopwords_and_2      42
pos_label                          0 
dtype: int64

In [95]:
len(df)

47783

In [82]:
# dividing into positive and negative dfs
df_pos = df.loc[df['pos_label'] == 1]

In [83]:
len(df_pos)

23000

In [84]:
df_neg = df.loc[df['pos_label'] == 0]

In [85]:
len(df_neg)

24783

In [72]:
get_tokens_frequency_df(df['tweet_without_stopwords2'])

TypeError: doc2bow expects an array of unicode tokens on input, not a single string